# xgboostを用いて、スプライスされやすい領域とそうでない領域の配列の学習を行う

**目的:**<br>
スプライシングを受けやすいイントロン領域とそうではないイントロン領域の二群に分割する<br>
各イントロンの両側n塩基の塩基配列を取得し、学習データとする<br>
教師あり学習であるXGBoostアルゴリズムを用いて、この二群のを分離させるような特徴的な塩基配列部位を同定する<br>
<br>

**略語**<br>
HAVSR: highly variable alternatively spliced<br>
LAVSR: Lowly variable alternatively spliced<br><br>

**注意点:**<br>
XGBClassifierのインスタンスおよびtrain_test_split関数のrandom stateを変更すると結果が異なる


## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

In [ ]:
## 各種変数の設定

# 5-prime側の側の末端の塩基
left_edge = gbk.exon_list()[42][0]

# 3-prime側の側の末端の塩基
right_edge = gbk.exon_list()[219][1]

# XGBoostに関する変数の設定
early_stopping_rounds = 10
learning_rate = 0.01
max_depth = 8
x_random_state = 1
s_random_state =  0
test_size = 0.2


In [ ]:
# 
gbk.transcript_variants()

In [ ]:
# 各種バリアントのIDを取得
vars = gbk.get_mrna_ids()

In [ ]:
# すべてのイントロンの始点終点のリストを作成
all_introns_list = []
for i in vars:
    g = Seq_count()
    g.read_gbk('../data/gbk/human_ttn.gb')
    g.set_mrna_id(i)
    for j in g.intron_list():
        all_introns_list.append(j)
introns_list = []
for i in all_introns_list:
    if not i in introns_list:
        introns_list.append(i)
introns_list.sort(key=lambda x: x[0])
introns_list = [[i,j] for i,j in introns_list if i > left_edge and j < right_edge]
introns_list

In [ ]:
# すべてのエクソンの始点終点のリストを作成
all_exons_list = []
for i in vars:
    g = Seq_count()
    g.read_gbk('../data/gbk/human_ttn.gb')
    g.set_mrna_id(i)
    for j in g.exon_list():
        all_exons_list.append(j)
exons_list = []
for i in all_exons_list:
    if not i in exons_list:
        exons_list.append(i)
exons_list.sort(key=lambda x: x[0])

In [ ]:
exons_list = [[i,j] for i,j in exons_list if i < right_edge and j > left_edge]

In [ ]:
possible_intron_combinations = []
for i, j in itertools.combinations(exons_list, 2):
    possible_intron_combinations.append([i[1], j[0]])
possible_intron_combinations

In [ ]:
seq = gbk.gDNA_seq()
for i in introns_list[-5:]:
    print(seq[i[0]:i[0]+10], seq[i[1]-10:i[1]])


In [ ]:
n = 15
existent_intron_combinations = []
non_existent_intron_combinations = []
for i in possible_intron_combinations:
    if i in introns_list:
        existent_intron_combinations.append(seq[i[0]:i[0]+n] + seq[i[1]-n:i[1]])
    else:
        non_existent_intron_combinations.append(seq[i[0]:i[0]+n] + seq[i[1]-n:i[1]])

In [ ]:
# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences_existent = np.array([one_hot_encode(seq) for seq in existent_intron_combinations])
encoded_sequences_non_existent = np.array([one_hot_encode(seq) for seq in non_existent_intron_combinations])

In [ ]:
# 学習データとしてワンホとエンコーディングされた配列を結合しXとする
# ラベルをyとして結合する
X = pd.DataFrame(np.concatenate([encoded_sequences_existent, encoded_sequences_non_existent]))
y = pd.DataFrame(np.concatenate([np.ones(len(encoded_sequences_existent)), np.zeros(len(encoded_sequences_non_existent))]))

# XGBoostのインスタンス化
model_l = XGBClassifier(early_stopping_rounds=early_stopping_rounds, learning_rate=learning_rate, max_depth=max_depth, random_state=x_random_state)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=s_random_state)

# データをモデルにfitさせる
eval_set = [(X_test, y_test)]
model_l.fit(X_train, y_train, eval_set=eval_set, verbose=False)    

In [ ]:
# logomakerを用いてモチーフを可視化
base_df = pd.DataFrame(model_l.feature_importances_.reshape(n*2,4))
base_df.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(base_df, shade_below=.5, fade_below=.5)

In [ ]:
vmin, vmax = 0, 1

# ヒートマップの描画
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 2))

# 最初のヒートマップ
l_df = pd.DataFrame([list(i) for i in existent_intron_combinations])
sns.heatmap(l_df.apply(pd.Series.value_counts).fillna(0).astype(int)/len(existent_intron_combinations), ax=ax1, cmap="viridis", vmin=vmin, vmax=vmax, cbar=False)
ax1.set_title("Left edge of introns in HVAS regions")

# 2つ目のヒートマップ
r_df = pd.DataFrame([list(i) for i in non_existent_intron_combinations])
cax = fig.add_axes([0.92, 0.12, 0.02, 0.76])  # カラーバーの位置とサイズを調整
sns.heatmap(r_df.apply(pd.Series.value_counts).fillna(0).astype(int)/len(non_existent_intron_combinations), ax=ax2, cmap="viridis",vmin=vmin, vmax=vmax, cbar_ax=cax)
ax2.set_title("Right edge of introns in HVAS regions")

# Figureの表示
fig.show()
